# Merge MERIT Hydro flow direction grid
Merges the individual files into a single large GeoTIFF.

In [1]:
import os
import sys
import shutil
from osgeo import gdal
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path   = cs.read_from_config(config_file,'data_path')
merit_path  = cs.read_from_config(config_file,'merit_path')
subset_area = cs.read_from_config(config_file,'merit_subset_area')

In [4]:
# Convert subsetting string into a list of four floats
subset_coor = list(map(float,subset_area.split(',')))

### Define data locations

In [5]:
# Construct the download location
download_folder = Path(data_path) / merit_path / 'flow_direction_grid'

In [6]:
# Name of the merged GeoTIFF
merged_file = 'merit_hydro_dir.tif'

### Merge the data into a GeoTIFF
The MERIT Hydro downloads cover a larger area than we are interested in. To save disk space we subset the download region to a smaller region before merging the individual `.tif` files into a single large one.

In [7]:
# Find the file names
all_files = []
for dir_path, dir_names, file_names in os.walk(download_folder):
    for file_name in file_names:
        if file_name.endswith('.tif'): # ensure we don't accidentally get .aux files from QGIS or something similar
            all_files.append(os.path.join(dir_path,file_name))

In [8]:
# Convert subsetting area into a usable GDAL setting
# subset_area = [lon_min, lon_max, lat_min, lat_max]
# GDAL window = [ulx, uly, lrx, lry]; [upper left x, upper left y, lower right x, lower right y]
# Mapping:
#   ulx = lon_min = subset_area[0]
#   uly = lat_max = subset_area[3]
#   lrx = lon_max = subset_area[1]
#   lry = lat_min = subset_area[2]
window = [subset_coor[0], subset_coor[3], subset_coor[1], subset_coor[2]]

In [9]:
cs.merge_merit_downloads_into_area_of_interest(all_files, str(download_folder/merged_file), window)

### Delete the individual GeoTIFF files to save space

In [10]:
# Structure to delete:
# - folder1
#    |- file1
#    |- file2
#    |- ..
# - folder2
#    |- ..

# Loop over all individual files and ...
for file in all_files:
    
    # Find the parent folder the file is in
    remove_this = os.path.dirname(file) 
    
    # Check if this folder still exists and ...
    if os.path.isdir(remove_this):
        shutil.rmtree(remove_this) # Remove the folder and everything in it if so